# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [31]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. 
'''

## 2. Generate the mathematical model

In [48]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [49]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*   **Patients:** i ∈ I = {1, 2, ..., n} (where n = 17)
*   **Proton Fractions:** j ∈ J = {0, 1, ..., 15} 

### Parameters:

*   **C:** Total proton fraction capacity (C = 100)
*   **BED<sub>i</sub>(j, 15-j):** Biological Equivalent Dose for patient i with j proton fractions and (15-j) photon fractions (obtained from "ProblemData.csv")

### Decision Variables:

*   **x<sub>ij</sub>:** Binary variable equal to 1 if patient i receives j proton fractions, and 0 otherwise.

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_(i∈I) ∑_(j∈J) BED_i(j, 15-j) * x_ij
```

### Constraints:

1.  **Proton Capacity Constraint:** The total number of proton fractions used cannot exceed the capacity C.
    ```
    ∑_(i∈I) ∑_(j∈J) j * x_ij ≤ C
    ```
2.  **Treatment Plan Constraint:** Each patient must receive exactly 15 fractions (either proton or photon).
    ```
    ∑_(j∈J) x_ij = 1, ∀ i ∈ I
    ```
3.  **Binary Variable Constraint:** 
    ```
    x_ij ∈ {0, 1}, ∀ i ∈ I, j ∈ J
    ```

### Model Summary:

This integer linear programming model aims to maximize the total BED score for all patients while adhering to the proton capacity constraint and ensuring each patient receives a complete treatment plan. 

The model uses binary decision variables to determine the optimal allocation of proton fractions to each patient. By solving this model, you can determine which patients should receive proton therapy and the number of proton fractions they should receive to achieve the best possible treatment outcomes within the available resources. 


## 3. Generate the pyomo code

In [50]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [51]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data from "ProblemData.csv")
BED = {
    (1, 0, 15): 70,
    (1, 1, 14): 72,
    # ... (add BED values for all patients and fraction combinations)
    (17, 15, 0): 85
}

# Define the model
model = pyo.ConcreteModel()

# Sets and indices
model.I = pyo.Set(initialize=range(1, 18))  # Patients (n = 17)
model.J = pyo.Set(initialize=range(16))  # Proton Fractions (0 to 15)

# Parameters
model.C = pyo.Param(initialize=100)  # Total proton fraction capacity

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= model.C
model.capacity_constraint = pyo.Constraint(rule=capacity_rule)

def treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.treatment_plan_constraint = pyo.Constraint(model.I, rule=treatment_plan_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in model.I:
    for j in model.J:
        if model.x[i, j].value > 0.5:
            print(f"Patient {i} receives {j} proton fractions.")

print(f"Total BED Score: {model.objective()}")
```

**Explanation:**

1. **Sample Data:** The `BED` dictionary is where you would replace the sample data with actual values from your "ProblemData.csv" file.
2. **Sets and Indices:** We define sets for patients (`model.I`) and proton fractions (`model.J`).
3. **Parameters:** The proton capacity `model.C` is defined as a parameter.
4. **Decision Variables:** The binary variable `model.x[i, j]` represents whether patient `i` receives `j` proton fractions.
5. **Objective Function:** The objective function maximizes the total BED score, calculated as the sum of BED values for each patient and fraction combination, multiplied by the corresponding decision variable.
6. **Constraints:** 
    - **Capacity Constraint:** Ensures the total number of proton fractions used doesn't exceed the capacity.
    - **Treatment Plan Constraint:** Makes sure each patient receives exactly 15 fractions (proton or photon). 
7. **Solve the Model:** We use the `glpk` solver to solve the model. You can choose other solvers as needed.
8. **Print Results:** The code prints the optimal solution, showing the number of proton fractions assigned to each patient and the total BED score achieved. 


## 4. Input problem data and try running the generated code

In [54]:
import pyomo.environ as pyo

# Sample Data (replace with actual data from "ProblemData.csv")
# BED = {
#     (1, 0, 15): 70,
#     (1, 1, 14): 72,
#     # ... (add BED values for all patients and fraction combinations)
#     (17, 15, 0): 85
# }

#LOADING THE DATA DONE BY HUMAN
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j, 15-j)] = dose
#END

# Define the model
model = pyo.ConcreteModel()

# Sets and indices
model.I = pyo.Set(initialize=range(1, 18))  # Patients (n = 17)
model.J = pyo.Set(initialize=range(16))  # Proton Fractions (0 to 15)

# Parameters
model.C = pyo.Param(initialize=100)  # Total proton fraction capacity

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= model.C
model.capacity_constraint = pyo.Constraint(rule=capacity_rule)

def treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.treatment_plan_constraint = pyo.Constraint(model.I, rule=treatment_plan_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in model.I:
    for j in model.J:
        if model.x[i, j].value > 0.5:
            print(f"Patient {i} receives {j} proton fractions.")

print(f"Total BED Score: {model.objective()}")

Optimal Solution:
Patient 1 receives 15 proton fractions.
Patient 2 receives 8 proton fractions.
Patient 3 receives 3 proton fractions.
Patient 4 receives 0 proton fractions.
Patient 5 receives 5 proton fractions.
Patient 6 receives 0 proton fractions.
Patient 7 receives 4 proton fractions.
Patient 8 receives 13 proton fractions.
Patient 9 receives 4 proton fractions.
Patient 10 receives 5 proton fractions.
Patient 11 receives 6 proton fractions.
Patient 12 receives 0 proton fractions.
Patient 13 receives 5 proton fractions.
Patient 14 receives 0 proton fractions.
Patient 15 receives 10 proton fractions.
Patient 16 receives 10 proton fractions.
Patient 17 receives 12 proton fractions.
Total BED Score: 8.239999999999998


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [55]:
print(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*   **Patients:** i ∈ I = {1, 2, ..., n} (where n = 17)
*   **Proton Fractions:** j ∈ J = {0, 1, ..., 15} 

### Parameters:

*   **C:** Total proton fraction capacity (C = 100)
*   **BED<sub>i</sub>(j, 15-j):** Biological Equivalent Dose for patient i with j proton fractions and (15-j) photon fractions (obtained from "ProblemData.csv")

### Decision Variables:

*   **x<sub>ij</sub>:** Binary variable equal to 1 if patient i receives j proton fractions, and 0 otherwise.

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_(i∈I) ∑_(j∈J) BED_i(j, 15-j) * x_ij
```

### Constraints:

1.  **Proton Capacity Constraint:** The total number of proton fractions used cannot exceed the capacity C.
    ```
    ∑_(i∈I) ∑_(j∈J) j * x_ij ≤ C
    ```
2.  **Treatment Plan Constraint:** Each patient must receive exactly 15 fractions (either proton or photon).
    ```
    ∑

In [56]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data from "ProblemData.csv")
BED = {
    (1, 0, 15): 70,
    (1, 1, 14): 72,
    # ... (add BED values for all patients and fraction combinations)
    (17, 15, 0): 85
}

# Define the model
model = pyo.ConcreteModel()

# Sets and indices
model.I = pyo.Set(initialize=range(1, 18))  # Patients (n = 17)
model.J = pyo.Set(initialize=range(16))  # Proton Fractions (0 to 15)

# Parameters
model.C = pyo.Param(initialize=100)  # Total proton fraction capacity

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= model.C
model.capacity_constraint = pyo.Constraint(rule=capacity_rule)

def tr